### 5.1 함수를 객체처럼 다루기

In [1]:
def factorial(n):
    '''return n!'''
    return 1 if n < 2 else n * factorial(n-1)

In [2]:
factorial(42)

1405006117752879898543142606244511569936384000000000

In [3]:
factorial.__doc__

'return n!'

In [4]:
type(factorial)

function

In [5]:
fact = factorial

In [6]:
fact

<function __main__.factorial(n)>

In [7]:
fact(5)

120

In [8]:
map(factorial, range(11))

In [9]:
list(map(factorial, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

- 일급 함수가 있으면 함수형 스타일로 프로그래밍할 수 있다.
- 함수형 프로그래밍의 특징 중 하나가 고위함수

### 5.2 고위함수

- 함수를 인수로 받거나 함수를 결과로 반환하는 함수를 고위함수(High-order function)이라고 한다.
- map(), list.sort(), sorted()함수 등등 존재.

In [10]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']

In [11]:
sorted(fruits, key = len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

- map(), filter(), reduce() 등의 고위함수는 여전히 존재하지만 대부분의 경우 더 나은 방법이 있다.

###### 5.2.1 map(), filter(), reduce()의 대안

In [12]:
list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [14]:
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [15]:
list(map(factorial, filter(lambda n : n%2, range(6))))

[1, 6, 120]

In [16]:
[factorial(n) for n in range(6) if n%2]

[1, 6, 120]

- map(), filter()는 제너레이터(일종의 반복 가능 객체)를 반환하므로, 제너레이터 표현식이 이 함수들을 직접 대체한다.

In [17]:
from functools import reduce
from operator import add

In [18]:
reduce(add, range(100))

4950

In [19]:
sum(range(100))

4950

###### 5.3 익명함수

- lambda 키워드는 익명함수를 생성한다.
- 그렇지만 파이썬의 단순한 구문이 람다 함수의 본체가 순순한 표현식으로만 구성되도록 제한한다.
- 즉, 람다 본체에서는 할당문이나 while, try 등의 파이썬 문장을 사용할 수 없다.

In [20]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']

In [21]:
sorted(fruits, key = lambda word : word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

###### 5.4 일곱가지 맛의 콜러블 객체

In [22]:
abs, str, 14

(<function abs(x, /)>, str, 14)

In [23]:
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

###### 5.5 사용자 정의 콜러블형

In [24]:
import random

class BingoCage:
    
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        return self.pick()

In [25]:
bingo = BingoCage(range(3))

In [29]:
bingo.pick()

LookupError: pick from empty BingoCage

In [30]:
callable(bingo)

True

###### 5.6 함수 인트로스펙션

In [31]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [32]:
class C: pass

In [33]:
obj = C()

In [34]:
def func(): pass

In [35]:
sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

###### 5.7 위치 매개변수에서 키워드 전용 매개변수까지

In [36]:
def tag(name, *content, cls = None, **attrs):
    '''하나 이상의 HTML 태그를 생성한다.'''
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s = "%s"' % (attr, value) for attr, value in sorted(attrs.items()))
    else :
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [38]:
tag('br')

'<br />'

In [39]:
tag('p', 'hello')

'<p>hello</p>'

In [40]:
def f(a, *, b):
    return a, b

In [41]:
f(1, b=2)

(1, 2)

###### 5.8 매개변수에 대한 정보 읽기

In [44]:
import bobo

In [45]:
@bobo.query('/')
def hello(person):
    return 'Hello %s!' % person

In [46]:
def clip(text, max_len=80):
    """max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스트를 반환한다.
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

In [49]:
import inspect
sig = inspect.signature(tag)

my_tag = {'name':'img', 'title':'Sunset Boulevard',
         'src':'sunset.jpg', 'cls':'framed'}
bound_args = sig.bind(**my_tag)
print(bound_args)

for name, value in bound_args.arguments.items():
    print(f'{name} = {value}')


del my_tag['cls']
bound_args = sig.bind(**my_tag)

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>
name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}
